In [1]:
using Pkg
Pkg.add("ProgressBars")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
using Random, LinearAlgebra, Plots
# bring packages into main namespace
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV                    # tools for working with CSV files
using Plots, Random, LinearAlgebra, Statistics, SparseArrays
using ProgressBars
include("proxgrad.jl")
pyplot()

Plots.PyPlotBackend()

In [3]:
dataset = CSV.read("Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv")
dataset = dropmissing(dataset)
dataset[:when] = dataset[:Year] .+ ((dataset[:quarter] .-1) ./ 4)
n = size(dataset,1)
r = .1
cuttoff = Int(round(n*r))
idxs = rand(1:n,n)

# We need to hold out a test dataset to report final results on, that is not used at all for model selection.
# Within k-fold cross validation, train and val datasets will be formed from this main train dataset. 
test_dataset  = dataset[idxs[1:cuttoff],:]
train_dataset = dataset[idxs[cuttoff+1:n],:]

n_train = size(train_dataset,1)
n_test = size(test_dataset,1)

@assert  n_train + n_test == n
@show n_train, n_test
head(dataset)

(n_train, n_test) = (181253, 20139)


6×24 DataFrame. Omitted printing of 19 columns
│ Row │ tbl      │ Year  │ quarter │ citymarketid_1 │ citymarketid_2 │
│     │ String   │ Int64 │ Int64   │ Int64          │ Int64          │
├─────┼──────────┼───────┼─────────┼────────────────┼────────────────┤
│ 1   │ Table 1a │ 2010  │ 1       │ 34614          │ 33195          │
│ 2   │ Table 1a │ 1998  │ 4       │ 30189          │ 31703          │
│ 3   │ Table 1a │ 1998  │ 4       │ 30198          │ 30852          │
│ 4   │ Table 1a │ 2009  │ 3       │ 32211          │ 32575          │
│ 5   │ Table 1a │ 1993  │ 4       │ 30255          │ 30852          │
│ 6   │ Table 1a │ 2010  │ 4       │ 33198          │ 32575          │

In [4]:
# Do k-fold cross validation and return the average error_metric on the validation set accross the k folds.
function cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(dataset,1)
    r = Int(round(n / k))
    idxs = rand(1:n,n) # to shuffle the dataset
    error = 0
    for i in (1:k)
        val_idxs = r*(i-1)+1:min(r*i,n)
        tr_low = 1:r*(i-1)
        tr_high = r*i+1:n
        if (i == 1)
            tr_idxs = tr_high
        elseif (i == k)
            tr_idxs = tr_low
        else
            tr_idxs = [tr_low ; tr_high ]
        end
        # @show i, val_idxs
        # @show tr_low, tr_high
        X_tr = X[idxs[tr_idxs],:]
        y_tr = y[idxs[tr_idxs]]
        
        X_val = X[idxs[val_idxs],:]
        y_val = y[idxs[val_idxs]]
        
        w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
        ŷ_val = X_val * w
        # @show size(y_val)
        # @show size(ŷ_val)
        error += error_metric(ŷ_val, y_val)
    end
    return error / k
end

function extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(X,1)
    
    tr_idxs = dataset[:when].< 2013.5
    val_idxs = dataset[:when].>= 2013.5
    
    X_tr = X[tr_idxs,:]
    y_tr = y[tr_idxs]
    n_tr = size(X_tr,1)

    X_val = X[val_idxs,:]
    y_val = y[val_idxs]
    n_val = size(X_val,1)
    
    @assert n_tr + n_val == n

    w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
    ŷ_val = X_val * w
    # @show size(y_val)
    # @show size(ŷ_val)
    
    return error_metric(ŷ_val, y_val)
end

function train_and_validate(featurizer, loss, regularizer, stepsize, error_metric, mode; k=10, dataset=train_dataset)
    if (mode=="interpolation")
        return cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    else
        return extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    end
end

# For each model in models, do k-fold cross validation and calculate the average error_metric
# on the val set accross the k-folds.
# Each model in model is a tuple of the form (featurizer, loss, regularizer, stepsize),
# where a featurizer is a funciton that takes in a dataset and returns X,y. 
# Returns errors for each model, and the index of the best model
function test_models(models, error_metric,mode;k=10, dataset=train_dataset)
    errors = []
    for model in models
        error = train_and_validate(model...,error_metric,mode;k=k,dataset=dataset)
        errors = [errors; error]
    end
    i = argmin(errors)
    println("The best model is model ",i)
    return errors,i
end

test_models (generic function with 1 method)

# Feature Engineering and Selection

In [5]:
cat_labels = [:carrier_lg, :carrier_low, :airportid_1, :airportid_2]
cat_labels_sub = [:carrier_lg, :carrier_low]

#Sets of all categories in a particular column
cats_sets = [unique(dataset[:, label]) for label in cat_labels]
cats_sets_sub = [unique(dataset[:, label]) for label in cat_labels_sub]

"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(length(column), length(cats))
    for i = 1:length(column)
        for j =1:length(cats)
            if column[i] === cats[j]
                result[i, j] = 1
            end    
        end
    end
    result
end

onehot

In [6]:
labels_real = [
    :when, 
    :citymarketid_1, 
    :citymarketid_2,
    :airportid_1,
    :airportid_2,
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]
labels_real_sub = [
    :when, 
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]

7-element Array{Symbol,1}:
 :when
 :nsmiles
 :passengers
 :large_ms
 :fare_lg
 :lf_ms
 :fare_low

In [7]:
# Baseline featurizer
function feats_0(dataset)
    X = [dataset[:when] dataset[:airportid_1] dataset[:airportid_2] ones(size(dataset,1))]
    y = dataset[:fare]
    return X,y
end

# Only numeric features (treating airportid and cityid as numeric)
function feats_1(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    X = hcat(real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as numerical)
function feats_2(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    cat_vals = hcat([onehot(dataset[:, cat_labels_sub[i]], cats_sets_sub[i]) for i in 1:size(cat_labels_sub, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as categorical)
function feats_3(dataset)
    real_vals = convert(Matrix, dataset[labels_real_sub])
    cat_vals = hcat([onehot(dataset[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

MSE(L1,L2) = sum((L1.-L2).^2) / size(L1,1)

MSE (generic function with 1 method)

In [8]:
models_0 = [
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9), # best model 
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]
errors,i = test_models(models_0,MSE,"extrapolation")

The best model is model 159


(Any[6582.096290076168, 6582.096304890767, 6582.096319705367, 6582.096275261576, 6582.097505107875, 6582.098734954792, 6582.099964802323, 6582.096275261576, 7316.674645830166, 7316.864768530977  …  13436.144107169946, 13432.404174482295, 5256.456469479519, 5256.438245641649, 5256.433967532269, 5256.453070909702, 5256.374440871506, 5256.388407658592, 5256.23090055887, 5256.453070909702], 159)

In [9]:
models_1 = [
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5), # best model
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_1,MSE,"extrapolation")

The best model is model 90


(Any[6352.324677680868, 6352.324680691043, 6352.324683701218, 6352.324674670692, 6352.326029932989, 6352.327385197298, 6352.328740463638, 6358.409622049468, 7174.654975821109, 7174.675709560015  …  13147.80970926749, 13148.704596992553, 5021.638550620819, 5021.661893600838, 5021.614294059494, 5021.641111252579, 5022.070180964043, 5022.567666451634, 5023.084672806755, 5029.091799371462], 90)

In [10]:
models_2 = [
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5), # best model
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_2,MSE,"extrapolation")

The best model is model 90


(Any[6352.324671645066, 6352.32467465524, 6352.324677665415, 6352.324668634889, 6352.326024181513, 6352.327379704241, 6352.328735236036, 6358.409613633957, 7174.654969428585, 7174.675703167518  …  13147.80970926749, 13148.704571244103, 5021.638534806242, 5021.661877785567, 5021.614278245135, 5021.641095437711, 5022.070180964043, 5022.567666451634, 5023.084672806755, 5029.107103674152], 90)

In [11]:
models_3 = [
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7), # best model
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_3,MSE,"extrapolation")

The best model is model 124


(Any[2727.762320156534, 2727.769531128684, 2727.776742071282, 2727.7551091547975, 2727.804214033805, 2727.853263025222, 2727.9022954819325, 2752.3913109648547, 3768.5071108490733, 3769.497015583217  …  7560.128161546006, 7475.4098441078795, 2142.090299044397, 2143.237583231831, 2144.4313861631053, 2141.03195043949, 2137.348519571062, 2134.222046182998, 2163.73416015301, 2315.3069245606903], 124)

In [14]:
best_models = [
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7) 
]

4-element Array{Tuple{Function,QuantileLoss,Regularizer,Float64},1}:
 (feats_0, QuantileLoss(5.517150060964508e-6, RealDomain(), 0.75), OneReg(0.75), 0.9)
 (feats_1, QuantileLoss(5.517150060964508e-6, RealDomain(), 0.75), QuadReg(0.5), 0.5)
 (feats_2, QuantileLoss(5.517150060964508e-6, RealDomain(), 0.75), QuadReg(0.5), 0.5)
 (feats_3, QuantileLoss(5.517150060964508e-6, RealDomain(), 0.75), ZeroReg(), 0.7)

In [15]:
errors_best,i_best = test_models(best_models,MSE,"extrapolation")

The best model is model 4


(Any[5256.23090055887, 5007.7988209247005, 5007.798803394528, 1781.7676151903313], 4)